# Using memory to have an Interactive Conversational Chatbot

In this notebook, we use Langchain to add memory to the LLM chatbot.  
The memory helps the LLM to keep track of the conversation and answer based on the context.  

**Case studys:** 
- Technical support
- Customer support
- Healthcare consultation

We are going to covers the following:
- Why memory is required?
- Complete memory recording
- Window based memory
- Token based memory
- Summary based memory
 

In [2]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("..")

In [3]:
import pprint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [4]:
# A function for printing nicely
def nprint(text, indent=2):
    pp = pprint.PrettyPrinter(indent=indent)
    pp.pprint(text)

# Loading Parameters

In [5]:
modelID = "gpt-3.5-turbo"

# Technical support as a chatbot technician
Here the LLM chatbot gives technical support to the customer to fix it's problem.  
We use system context to customize the chatbot response.


In [6]:
# defining the chatbot model and system role
from langchain_openai import ChatOpenAI
sys_content = """
You are an expert technician who gives exact, clear, and short steps to solving a problem.\
Give the instruction in seperate steps.
"""
QAchat = ChatOpenAI(temperature=0, model=modelID)

A simple question to get some intstruction steps.

In [37]:
question = "What do I do if my screen freezes?"
prompt = [
    (
        "system",
        f'{sys_content}',
    ),
    ("human", f'{question}'),
]
prompt
completion = QAchat.invoke(prompt)
nprint(completion.content)

('1. Press and hold the power button on your device until it shuts down.\n'
 '2. Wait a few seconds, then press the power button again to turn it back '
 'on.\n'
 '3. If the screen continues to freeze, try restarting the device in safe mode '
 "(refer to your device's manual for instructions).\n"
 '4. If the issue persists, consider performing a factory reset or seeking '
 'professional help.')


## Using no memory
Let's see if LLM can remember the steps by default.

In [28]:
completion = QAchat.invoke("Please explain the 2nd step in more detail")
nprint(completion.content)

('The second step is to gather all the necessary materials and resources '
 'needed to complete the task or project. This may include tools, equipment, '
 'supplies, information, and any other resources that are essential for the '
 'successful completion of the task. It is important to ensure that all '
 'necessary materials are readily available and easily accessible before '
 'starting the task to avoid any delays or interruptions. Additionally, it is '
 'important to double-check that all materials are in good working condition '
 'and that there are no missing or damaged items that could hinder progress. '
 'By thoroughly preparing and organizing all necessary materials, you can '
 'streamline the process and increase the likelihood of a successful outcome.')


## Using memory: Complete chat history

In [7]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Defining the buffer memory for the chatbot
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=QAchat, 
    memory = memory,
    verbose=True
)

In [87]:
prompt = f"""{sys_content}
Answer the following question:  
{question} 
"""
completion = conversation.predict(input=prompt)
nprint(completion)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 
You are an expert technician who gives exact, clear, and short steps to solving a problem.Give the instruction in seperate steps.

Answer the following question:  
What do I do if my screen freezes? 

AI:

> Finished chain.
('Sure! Here are the steps to follow if your screen freezes:\n'
 '\n'
 'Step 1: Try pressing Ctrl + Alt + Delete on your keyboard to bring up the '
 'Task Manager.\n'
 'Step 2: In the Task Manager, look for any unresponsive programs or '
 'processes.\n'
 'Step 3: Select the frozen program or process and click on "End Task" to '
 'force it to close.\n'
 'Step 4: If the screen is still frozen, try restarting your computer by '
 'holding down 

Let's see if it can remember the steps

In [53]:
completion = conversation.predict(input="Explain the step 2 in more detail")
nprint(completion)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 
You are an expert technician who gives exact, clear, and short steps to solving a problem.Give the instruction in seperate steps.

Answer the following question:  
What do I do if my screen freezes? 

AI: Sure! Here are the steps to follow if your screen freezes:

Step 1: Try pressing Ctrl + Alt + Delete on your keyboard to bring up the Task Manager.
Step 2: In the Task Manager, look for any unresponsive programs or processes and end them.
Step 3: If that doesn't work, try restarting your computer by holding down the power button until it shuts off.
Step 4: Once your computer has restarted, check for any updates or software issues that may have caused the freez

Let's try a question from the first answer

In [95]:
completion = conversation.predict(input="Explain the step 3 in more detail")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 
You are an expert technician who gives exact, clear, and short steps to solving a problem.Give the instruction in seperate steps.

Answer the following question:  
What do I do if my screen freezes? 

AI: Sure! Here are the steps to follow if your screen freezes:

Step 1: Try pressing Ctrl + Alt + Delete on your keyboard to bring up the Task Manager.
Step 2: In the Task Manager, look for any unresponsive programs or processes.
Step 3: Select the frozen program or process and click on "End Task" to force it to close.
Step 4: If the screen is still frozen, try restarting your computer by holding down the power button until it shuts off.
Step 5: Once the computer 

In [96]:
nprint(completion)
print(f'\n Length of buffer: {len(memory.buffer)}')

('Certainly! In step 3, after you have identified the frozen program or '
 'process in the Task Manager, clicking on it to select it is essentially '
 'highlighting the specific program or process that is causing the screen '
 'freeze. Once you have selected it, clicking on the "End Task" button at the '
 'bottom right corner of the Task Manager window is like giving the command to '
 'forcibly close that particular program or process. This action is necessary '
 'to stop the unresponsive program from continuing to run and causing the '
 'screen to remain frozen. By ending the task, you are essentially terminating '
 "the program's operation, allowing your computer to potentially regain "
 'functionality and resolve the screen freeze issue.')

 Length of buffer: 4430


Looking at the complete memory content

In [99]:
nprint(memory.buffer)

('Human: \n'
 'You are an expert technician who gives exact, clear, and short steps to '
 'solving a problem.Give the instruction in seperate steps.\n'
 '\n'
 'Answer the following question:  \n'
 'What do I do if my screen freezes? \n'
 '\n'
 'AI: Sure! Here are the steps to follow if your screen freezes:\n'
 '\n'
 'Step 1: Try pressing Ctrl + Alt + Delete on your keyboard to bring up the '
 'Task Manager.\n'
 'Step 2: In the Task Manager, look for any unresponsive programs or '
 'processes.\n'
 'Step 3: Select the frozen program or process and click on "End Task" to '
 'force it to close.\n'
 'Step 4: If the screen is still frozen, try restarting your computer by '
 'holding down the power button until it shuts off.\n'
 'Step 5: Once the computer has restarted, check if the screen is working '
 'properly again.\n'
 '\n'
 'I hope these steps help you resolve the issue with your frozen screen! Let '
 'me know if you need any further assistance.\n'
 'Human: Please explain each step in m

## Using memory: Partial chat history

Here, the memory module only holds the last few exchanges of the whole conversation.  
We also construct the LLM to ask for any missing or information that is beyond its memory window.   
**Use case:** Customer support.


In [52]:
#  Setting up the memory and loading it with sample conversation history

from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts.prompt import PromptTemplate
from langchain.prompts import ChatPromptTemplate

memory = ConversationBufferWindowMemory(k=2)

conversation_flow = [
    {"input": "Hi, I'm having an issue with my recent order.", "output": "I'm sorry to hear that. Can you please provide your order number?"},
    {"input": "It's 123456.", "output": "Thank you. What seems to be the problem with your order?"},
    {"input": "The item I received is damaged.", "output": "I'm sorry for the inconvenience. Would you like a replacement or a refund?"},
    # {"input": "I'd prefer a replacement.", "output": "I'll arrange for a replacement. Can you confirm your shipping address is the same as on the order?"},
    # {"input": "Yes, the shipping address is the same.", "output": "Great, I've processed the replacement order for you. You'll receive an email confirmation shortly. Could you please send a photo of the damaged item to our support email for our records?"},
    # {"input": "Sure, I can do that. What is the support email?", "output": "The support email is support@example.com. Once we receive the photo, we can expedite the replacement process. Is there anything specific you'd like us to check in the replacement item?"},
    # {"input": "Yes, please make sure it's properly packaged this time.", "output": "Absolutely, we'll ensure it's securely packaged. Do you need any assistance with returning the damaged item?"}
]

# Adding the conversation flow to the buffer
for conversation in conversation_flow:
    memory.save_context({'input':conversation["input"]},{'output': conversation["output"]})

# Looking at the memory buffer
nprint(memory.buffer)

("Human: It's 123456.\n"
 'AI: Thank you. What seems to be the problem with your order?\n'
 'Human: The item I received is damaged.\n'
 "AI: I'm sorry for the inconvenience. Would you like a replacement or a "
 'refund?')


Setting up the LLM assistant agent and its context

In [53]:
template = """
The following is a conversation between a human and an AI. \
The AI is a customer service agent who helps the customer to solve problems with their recent orders.\
The AI asks information from the customer where required. \
To send a replacement, AI needs to confirm if the shipping address is the same on the order. \
To process damaged items the AI needs a photo of the damaged item. \
Only ask one question at a time. \
The AI suggestions is in the framework of an ecommerce store and follows the standard conversation protocols of customer supports in such environments. 

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=QAchat,
    verbose=False,
    memory=memory,
)

Now proceeding with the conversation

In [54]:
completion = conversation.predict(input="I'd prefer a replacement.")
nprint(completion)
print('\n THE MEMORY BUFFER: \n')
nprint(memory.buffer)

('Thank you for letting us know. Can you please provide a photo of the damaged '
 'item so we can process the replacement for you?')

 THE MEMORY BUFFER: 

('Human: The item I received is damaged.\n'
 "AI: I'm sorry for the inconvenience. Would you like a replacement or a "
 'refund?\n'
 "Human: I'd prefer a replacement.\n"
 'AI: Thank you for letting us know. Can you please provide a photo of the '
 'damaged item so we can process the replacement for you?')


In [55]:
completion = conversation.predict(input="I sent it to the support email.")
nprint(completion)
print('\n THE MEMORY BUFFER: \n')
nprint(memory.buffer)

('Thank you for sending the photo. Just to confirm, is the shipping address on '
 'your order still the same for the replacement item to be sent to?')

 THE MEMORY BUFFER: 

("Human: I'd prefer a replacement.\n"
 'AI: Thank you for letting us know. Can you please provide a photo of the '
 'damaged item so we can process the replacement for you?\n'
 'Human: I sent it to the support email.\n'
 'AI: Thank you for sending the photo. Just to confirm, is the shipping '
 'address on your order still the same for the replacement item to be sent to?')


In [56]:
completion = conversation.predict(input="Yes, It's the same address.")
nprint(completion)
print('\n THE MEMORY BUFFER: \n')
nprint(memory.buffer)

('Great, thank you for confirming. We will process the replacement order for '
 'you. Is there anything else I can assist you with today?')

 THE MEMORY BUFFER: 

('Human: I sent it to the support email.\n'
 'AI: Thank you for sending the photo. Just to confirm, is the shipping '
 'address on your order still the same for the replacement item to be sent '
 'to?\n'
 "Human: Yes, It's the same address.\n"
 'AI: Great, thank you for confirming. We will process the replacement order '
 'for you. Is there anything else I can assist you with today?')


## Using memory: Summary of chat history

Here the memory module hold the summary of the chat history so far.   
**Use Case:** Healthcare - summarizing patient consultations

In [140]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=QAchat, max_token_limit=100)

# A sample conversation flow than can happen as a patient consultation
conversation_flow = [
    {"input": "I've been experiencing severe headaches for the past two weeks.", "output": "Can you describe the type of headache? Is it throbbing, constant, or does it come and go?"},
    {"input": "It's a constant dull ache that sometimes becomes throbbing.", "output": "Are there any other symptoms accompanying the headache, like nausea or sensitivity to light?"},
    {"input": "Yes, I often feel nauseous and I'm very sensitive to bright lights.", "output": "Have you taken any medication for these headaches?"},
    {"input": "I've been taking over-the-counter pain relievers, but they don't seem to help much.", "output": "Do you have a history of migraines or any other medical conditions?"},
    {"input": "I had migraines as a teenager, but they went away for a while. I also have high blood pressure.", "output": "Understood. I'll summarize this conversation for the doctor. Is there anything else you'd like to add?"},
    {"input": "No, that's all for now. Thank you.", "output": ""}
]

# Adding the conversation flow to the buffer
for conversation in conversation_flow:
    memory.save_context({'input':conversation["input"]},{'output': conversation["output"]})

Looking at how llm summarized the conversation

In [141]:
nprint(memory.buffer)

('System: The human mentions experiencing severe headaches for the past two '
 'weeks, describing them as a constant dull ache that sometimes becomes '
 'throbbing. The AI asks about accompanying symptoms like nausea and '
 'sensitivity to light, to which the human confirms. The AI inquires if the '
 'human has taken any medication for the headaches. The human responds that '
 "they have been taking over-the-counter pain relievers, but they don't seem "
 'to help much.\n'
 'AI: Do you have a history of migraines or any other medical conditions?\n'
 'Human: I had migraines as a teenager, but they went away for a while. I also '
 'have high blood pressure.\n'
 "AI: Understood. I'll summarize this conversation for the doctor. Is there "
 "anything else you'd like to add?\n"
 "Human: No, that's all for now. Thank you.\n"
 'AI: ')


Now asking specific questions about the conversation

In [142]:
conversation = ConversationChain(
    llm=QAchat, 
    memory = memory,
    verbose=False
)
questions = [
    "What are the patient's main symptoms?",
    "What medications has the patient taken?",
    "Does the patient have any relevant medical history?"
]
for question in questions:
    nprint(f"Question: {question}\n")
    completion = conversation.predict(input=question)
    nprint(completion)

"Question: What are the patient's main symptoms?\n"
("The patient's main symptoms are severe headaches that have been constant for "
 'the past two weeks, described as a dull ache that sometimes becomes '
 'throbbing. The patient also experiences nausea and sensitivity to light. '
 'They have a history of migraines as a teenager and currently have high blood '
 'pressure. The patient has been taking over-the-counter pain relievers, but '
 'they do not seem to help much.')
'Question: What medications has the patient taken?\n'
('The patient has been taking over-the-counter pain relievers for their '
 'headaches.')
'Question: Does the patient have any relevant medical history?\n'
('Yes, the patient has a history of migraines as a teenager and high blood '
 'pressure.')


# Conclusion

We saw that it is possible to use Langchain's memory module to keep track of the conversation. However, keeping all history in the memory can incur unnecessary load on the LLM.  
Based on the task and context, one can use the window based buffer memeory or summary based memory.  
Keep in mind that summary based memory can also cost additional costs on the LLM application.